In [1]:
import pandas as pd
import re

In [1]:
#files needed

In [2]:
folder = 'data/'
all_games = 'all_games_RWC2023.csv'
rubycology = 'Rugbycology_dataset_v2.csv'

In [3]:
rwc_2023_raw= pd.read_csv(folder+all_games)

# Clean the data if necessary

In [4]:
def clean_data_prepa(dataset):
    #look for missing data, there are none
    missing_values_sum = dataset.isna().sum() 
    print(missing_values_sum[missing_values_sum > 0] if any(missing_values_sum > 0) else 'No missing values')
    #look for spelling mistakes in teams name, all good
    team_A = list(dataset['Team A'].unique())
    team_B = list(dataset['Team B'].unique())
    teams = set(team_A + team_B)
    print(teams)
    dataset.Date = pd.to_datetime(dataset.Date)
    dataset[['Result A', 'Result B']] = dataset.apply(lambda row: pd.Series(['W' if row['Score A'] > row['Score B'] else ('D' if row['Score A'] == row['Score B'] else 'L'), 'W' if row['Score B'] > row['Score A'] else ('D' if row['Score B'] == row['Score A'] else 'L')]), axis=1)
    return dataset

In [8]:
rwc_clean = clean_data_prepa(rwc_2023_raw)
rwc_clean[['Metres A', 'Passes A']]

No missing values
{'Scotland', 'Samoa', 'Namibia', 'Japan', 'Italy', 'New Zealand', 'England', 'Chile', 'Tonga', 'Argentina', 'Ireland', 'Portugal', 'Uruguay', 'Romania', 'Fiji', 'Georgia', 'South Africa', 'Australia', 'France', 'Wales'}


,Metres A,Passes A
0,459,459
1,421,421
2,236,236
3,522,522
4,524,524
5,501,501
6,633,633
7,619,619
8,737,737
9,735,735


In [130]:
#split the dataset in rows for team A and team B
def create_team_df(df, letter):
    #create a df from the original df for only team A or B(letter).
    #output:df with all variables without the A or B
    contains_letter = [col for col in rwc_clean.columns if re.search(r' {0}(?: |$)'.format(letter), col)]
    contains_letter = ['Date', 'Game ID', 'Knock-out'] + contains_letter
    team = df[contains_letter]
    for col in team:
        team.rename(columns={col:  re.sub(r' {0}(?= |$)'.format(letter), '', col)}, inplace=True)

    return team
  
team_A = create_team_df(rwc_clean, 'A')
team_B = create_team_df(rwc_clean, 'B')
teams = pd.concat([team_A, team_B], ignore_index=True)

teams.to_csv('data/team_split.csv', index=False)

print(teams.head(5))

        Date  Game ID Knock-out         Team  Score  Possession - Overall  \
0 2023-10-28    87881       Yes  New Zealand     11                    60   
1 2023-10-27    87880       Yes    Argentina     23                    53   
2 2023-10-21    87879       Yes      England     15                    56   
3 2023-10-20    87878       Yes    Argentina      6                    43   
4 2023-10-15    87877       Yes       France     28                    60   

   Possession - 1st half  Possession - 2nd half  Tries  Conversions  ...  \
0                     63                     58      1            0  ...   
1                     45                     59      2            2  ...   
2                     52                     59      0            0  ...   
3                     46                     40      0            0  ...   
4                     59                     60      3            2  ...   

   Total Rucks  Lineouts Successful  Lineouts Thrown  Scrums Successful  \
0    

/tmp/ipykernel_226060/3667189210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team.rename(columns={col:  re.sub(r' {0}(?= |$)'.format(letter), '', col)}, inplace=True)
/tmp/ipykernel_226060/3667189210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team.rename(columns={col:  re.sub(r' {0}(?= |$)'.format(letter), '', col)}, inplace=True)
